In [31]:
from partition import Partition
from generators import PartitionClass
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, ConvLSTM2D, BatchNormalization, Conv3D
from keras.callbacks.callbacks import EarlyStopping, ReduceLROnPlateau

In [58]:
test_partition = training_partitions[12924]

In [90]:
partition = test_partition._next_oblak_step._next_oblak_step._next_oblak_step

In [91]:
partition

(21, 13, 3, 1, 1, 1)

In [92]:
partition._next_oblak_step

(21, 13, 3, 1, 1, 1)

In [93]:
test_partition

(15, 5, 3, 3, 3, 2, 2, 2, 1, 1, 1, 1, 1)

In [85]:
oblak = []
ar_parts = partition.ar_parts

In [86]:
max_u_chain_start_index = max(ar_parts, key=lambda x: 2 * x + sum(ar_parts[x]))
max_u_chain_value = 2 * max_u_chain_start_index + sum(ar_parts[max_u_chain_start_index])

if partition[0] >= max_u_chain_value:
    oblak.append(partition[0])
    partition = Partition(partition[1:])

else:
    partition = Partition(
        tuple([part - 2 for part in partition[:max_u_chain_start_index]])
        + partition[max_u_chain_start_index + len(ar_parts[max_u_chain_start_index]) :]
    )

    oblak.append(max_u_chain_value)

ar_parts = partition.ar_parts

In [88]:
oblak

[21]

In [60]:
test_partition.oblak

(21, 13, 5, 1)

# Training Data

In [28]:
n = 40

In [29]:
training_partitions = [p for p in PartitionClass(n).partitions]
training_matrices = [p.matrix for p in training_partitions]
training_durfees = [p._next_oblak_step.matrix for p in training_partitions]

X = np.array(training_matrices)
y = np.array(training_durfees)

X_train, X_test, y_train, y_test = train_test_split(X, y)

X_train = X_train.reshape(len(X_train), n, n, 1)
X_test = X_test.reshape(len(X_test), n, n, 1)
y_train = y_train.reshape(len(y_train), n, n, 1)
y_test = y_test.reshape(len(y_test), n, n, 1)

## Keras Model

In [32]:
model = Sequential()
model.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
input_shape=(None, 40, 40, 1),
padding="same", return_sequences=True))
model.add(BatchNormalization())

model.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
padding="same", return_sequences=True))
model.add(BatchNormalization())

model.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
padding="same", return_sequences=True))
model.add(BatchNormalization())

model.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
padding="same", return_sequences=True))
model.add(BatchNormalization())

model.add(Conv3D(filters=1, kernel_size=(3, 3, 3),
activation="sigmoid",
padding="same", data_format="channels_last"))
model.compile(loss="binary_crossentropy", optimizer="adadelta")

In [33]:
model.compile(loss="binary_crossentropy", optimizer="adadelta", metrics=['accuracy'])

# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [34]:
lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
es = EarlyStopping(monitor='val_loss', min_delta=.0001, patience=300, verbose=0, mode='auto', baseline=None, restore_best_weights=False)
cbs = [es, lr]

In [35]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=1000, callbacks=cbs, batch_size=100, workers = 4)

ValueError: Error when checking input: expected conv_lst_m2d_6_input to have 5 dimensions, but got array with shape (28003, 40, 40, 1)

In [38]:
def generate_movies(n_samples=1200, n_frames=15):
    row = 80
    col = 80
    noisy_movies = np.zeros((n_samples, n_frames, row, col, 1), dtype=np.float)
    shifted_movies = np.zeros((n_samples, n_frames, row, col, 1),
    dtype=np.float)

    for i in range(n_samples):
        # Add 3 to 7 moving squares
        n = np.random.randint(3, 8)

    for j in range(n):
        # Initial position
        xstart = np.random.randint(20, 60)
        ystart = np.random.randint(20, 60)
        # Direction of motion
        directionx = np.random.randint(0, 3) - 1
        directiony = np.random.randint(0, 3) - 1

    # Size of the square
    w = np.random.randint(2, 4)

    for t in range(n_frames):
        x_shift = xstart + directionx * t
        y_shift = ystart + directiony * t
        noisy_movies[i, t, x_shift - w: x_shift + w,
        y_shift - w: y_shift + w, 0] += 1

    # Make it more robust by adding noise.
    # The idea is that if during inference,
    # the value of the pixel is not exactly one,
    # we need to train the network to be robust and still
    # consider it as a pixel belonging to a square.
    if np.random.randint(0, 2):
        noise_f = (-1)**np.random.randint(0, 2)
        noisy_movies[i, t,
        x_shift - w - 1: x_shift + w + 1,
        y_shift - w - 1: y_shift + w + 1,
        0] += noise_f * 0.1

    # Shift the ground truth by 1
    x_shift = xstart + directionx * (t + 1)
    y_shift = ystart + directiony * (t + 1)
    shifted_movies[i, t, x_shift - w: x_shift + w,
    y_shift - w: y_shift + w, 0] += 1

    # Cut to a 40×40 window
    noisy_movies = noisy_movies[::, ::, 20:60, 20:60, ::]
    shifted_movies = shifted_movies[::, ::, 20:60, 20:60, ::]
    noisy_movies[noisy_movies >= 1] = 1
    shifted_movies[shifted_movies >= 1] = 1
    return noisy_movies, shifted_movies

In [45]:
generate_movies(n_samples = 100)[1].shape

(100, 15, 40, 40, 1)


## Next_Oblak_CNN

In [ ]:
n = 28

In [ ]:
training_partitions = PartitionClass(n).partitions
training_matrices = [p.matrix for p in training_partitions]
training_durfees = [p._next_oblak_step.matrix for p in training_partitions]

X = np.array(training_matrices)
y = np.array(training_durfees)

X_train, X_test, y_train, y_test = train_test_split(X, y)

X_train = X_train.reshape(len(X_train), n, n, 1)
X_test = X_test.reshape(len(X_test), n, n, 1)
y_train = y_train.reshape(*y_train.shape, -1)
y_test = y_test.reshape(*y_test.shape, -1)

In [ ]:
input_img = Input(shape=(n, n, 1))

x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# at this point the representation is (4, 4, 8) i.e. 128-dimensional

x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

model = Model(input_img, decoded)
model.summary()

In [ ]:
input_img = Input(shape=(n, n, 1))

x = Conv2D(16, (4, 4), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (4, 4), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (4, 4), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# at this point the representation is (4, 4, 8) i.e. 128-dimensional

x = Conv2D(8, (4, 4), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (4, 4), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (4, 4), activation='sigmoid', padding='same')(x)

model = Model(input_img, decoded)
model.summary()

In [ ]:
model.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
es = EarlyStopping(monitor='val_loss', min_delta=.001, patience=300, verbose=0, mode='auto', baseline=None, restore_best_weights=False)
cbs = [es, lr]

In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, callbacks=cbs, batch_size=100, workers = 4)

In [ ]:
for partition in PartitionClass(n).partitions:
    model_input = np.array(PartitionClass(n).partitions[0].matrix).reshape(-1, *(n, n, 1))
    prediction = Partition([x.sum() for x in model.predict(model_input)[0].round() if x.sum()])
    
    assert partition.oblak == prediction, (partition, partition.oblak, prediction)

In [ ]:
validation = [
    {
        'original': partition,
        'oblak': partition.oblak,
        'prediction': Partition([x.sum() for x in model.predict(np.array(partition.matrix).reshape(-1, *(n, n, 1)))[0].round() if x.sum()])
    }
 for partition in PartitionClass(n).partitions
    ]

In [ ]:
sorted(validation, key = lambda x: x['prediction'])[21]

## Validation

In [ ]:
model.evaluate(X_test, y_test, verbose=0)

In [ ]:
model_input = np.array(PartitionClass(n).partitions[0].matrix).reshape(-1, *(n, n, 1))
Partition([x.sum() for x in model.predict(model_input)[0].round() if x.sum()])


In [ ]:
for partition in PartitionClass(n).partitions:
    model_input = np.array(partition.matrix).reshape(-1, *(n, n, 1))
    prediction = label_binarizer.inverse_transform(model.predict(model_input))[0]
    
    assert partition.oblak == prediction, (partition.oblak, prediction)


# Feature Importance

In [ ]:
n = 30

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
import eli5
from eli5.sklearn import PermutationImportance

In [ ]:
def base_model():
    #create model
    model = Sequential()
    #add model layers
    model.add(Conv2D(64, kernel_size=2, activation="relu", input_shape=(n, n, 1)))
    model.add(Conv2D(32, kernel_size=2, activation="relu"))
    model.add(Flatten())
    model.add(Dense(units=len(y[0]), activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

def build_data(n):
    training_partitions = PartitionClass(n).partitions
    training_matrices = [p.matrix for p in training_partitions]
    training_durfees = [p.oblak for p in training_partitions]

    X = np.array(training_matrices)
    y = np.array(training_durfees)

    partition_encoder = dict(zip(set(y), range(len(set(y)))))
    y = np.array([partition_encoder[partition] for partition in y])
    partition_decoder = {v: k for k, v in partition_encoder.items()}

    label_binarizer = LabelBinarizer()
    y = label_binarizer.fit_transform(y)

    X_train, X_test, y_train, y_test = train_test_split(X, y)

    X_train = X_train.reshape(*X_train.shape, -1)
    X_test = X_test.reshape(*X_test.shape, -1)
    
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = build_data(n)

model = KerasRegressor(build_fn=base_model)

lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
es = EarlyStopping(monitor='val_loss', min_delta=.0001, patience=300, verbose=0, mode='auto', baseline=None, restore_best_weights=False)
cbs = [es, lr]
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, callbacks=cbs, batch_size=100, workers = 4)

In [ ]:
perm = PermutationImportance(model, random_state = 1).fit(X_train,y_train)
eli5.show_weights(perm, feature_names = X._traincolumns.tolist())

In [ ]:
len(X_train[0].flatten())

In [ ]:
model.loss


In [ ]:
model.fit(X_train, b
         )

In [ ]:
input_dim = model.input.shape.as_list()